In [1]:
import pandas as pd
import geopandas as gpd

In [ ]:
build_embeds = [
    '../data/lt_2020_out/lt_2020_combined.geojson',
    '../data/lt_2024_out/lt_2024_combined.geojson',
    '../data/ee_2020_out/ee_2020_combined.geojson',
    '../data/ee_2024_out/ee_2024_combined.geojson',
]
shelter_data = [
    '../data/LT/buildings/shelter_buildings_lt.geojson',
    '../data/LT/buildings/shelter_buildings_lt.geojson',
    '../data/EE/buildings/shelter_buildings_ee.geojson',
    '../data/EE/buildings/shelter_buildings_ee.geojson',
]
shelters_out = []

for embeds, shelts in zip(build_embeds, shelter_data):
    builds = gpd.read_file(embeds)[['type', 'Shape_Length', 'Shape_Area', 'resnet50_emb', 'geometry']]
    factual_shelters = gpd.read_file(shelts)[['geometry']]
    factual_shelters['is_shelter'] = 1
    
    if builds.crs != factual_shelters.crs:
        factual_shelters = factual_shelters.to_crs(builds.crs)
    
    shelters_embeds = gpd.sjoin(builds, factual_shelters, predicate='intersects', how='left').drop_duplicates()
    shelters_embeds['is_shelter'] = shelters_embeds['is_shelter'].fillna(0)
    shelters_out.append(shelters_embeds)

shelters_gdf = gpd.GeoDataFrame(pd.concat(shelters_out, ignore_index=True))
shelters_gdf.set_crs(shelters_out[0].crs, inplace=True)
shelters_gdf = shelters_gdf.drop('index_right', axis=1)

shelters_gdf.to_file('../data/lt_ee_data_for_model.geojson', driver='GeoJSON')